# Day9_4: 통계 이론 기초 - 정답 및 풀이

## 📚 학습 목표 복습

**Part 1: 기초 개념 - 대수의 법칙과 중심극한정리**
- 모집단, 표본, 모수, 통계량
- 대수의 법칙 (LLN)
- 중심극한정리 (CLT)
- 표준오차 (SE)

**Part 2: 심화 개념 - 검정력과 실험 설계**
- Type I/II 오류
- 검정력 (Power)
- 효과 크기 (Cohen's d)
- 표본 크기 결정

---

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("라이브러리 임포트 완료!")

---

## Q1. 표준오차 계산 ⭐

**문제**: 모집단 표준편차가 20이고 표본 크기가 100일 때 표준오차(SE)를 계산하세요.

```python
sigma = 20
n = 100
```

In [ ]:
# Q1 정답
sigma = 20  # 모집단 표준편차
n = 100     # 표본 크기

# 표준오차 공식: SE = sigma / sqrt(n)
se = sigma / np.sqrt(n)

print(f"모집단 표준편차 (sigma): {sigma}")
print(f"표본 크기 (n): {n}")
print(f"표준오차 (SE): {se}")

In [ ]:
# 검증
assert se == 2.0, "표준오차 계산 오류"
print("검증 통과! SE = 2.0")

### 💡 Q1 풀이 설명

**접근 방법**:
- 표준오차 공식 SE = σ/√n 적용

**핵심 개념**:
- 표준오차(Standard Error): 표본평균의 표준편차
- 표본 크기가 커질수록 표준오차 감소
- n이 4배 → SE가 절반

**실무 의미**:
- SE=2라는 것은 표본평균이 모평균에서 약 2만큼 오차날 수 있다는 의미
- 95% 신뢰구간: 모평균 ± 1.96×SE = 모평균 ± 3.92

---

## Q2. 대수의 법칙 시뮬레이션 ⭐

**문제**: 주사위를 2000번 던져서 누적 평균이 기댓값(3.5)에 수렴하는 것을 확인하세요.

In [ ]:
# Q2 정답
np.random.seed(42)

# 주사위 던지기 (1~6)
n_rolls = 2000
rolls = np.random.randint(1, 7, n_rolls)

# 누적 평균 계산
cumulative_means = np.cumsum(rolls) / np.arange(1, n_rolls + 1)

# 결과 확인
print("대수의 법칙: 주사위 던지기")
print("=" * 50)
print(f"주사위 기댓값: 3.5")
print(f"\n던진 횟수별 누적 평균:")
for n in [10, 50, 100, 500, 1000, 2000]:
    mean = cumulative_means[n-1]
    error = abs(mean - 3.5)
    print(f"  n={n:>4}: 평균={mean:.4f}, 오차={error:.4f}")

In [ ]:
# 시각화
df = pd.DataFrame({
    '던진 횟수': range(1, n_rolls + 1),
    '누적 평균': cumulative_means
})

fig = px.line(df, x='던진 횟수', y='누적 평균',
              title='대수의 법칙: 주사위 2000번 던지기')
fig.add_hline(y=3.5, line_dash="dash", line_color="red",
              annotation_text="기댓값 = 3.5")
fig.update_layout(height=400, width=700)
fig.show()

In [ ]:
# 검증: 최종 평균이 3.5에 가까운지 확인
final_mean = cumulative_means[-1]
assert abs(final_mean - 3.5) < 0.1, "대수의 법칙 미수렴"
print(f"검증 통과! 최종 평균 {final_mean:.4f}은 기댓값 3.5에 수렴")

### 💡 Q2 풀이 설명

**접근 방법**:
1. `np.random.randint(1, 7, n)`: 1~6 사이 정수 n개 생성
2. `np.cumsum()`: 누적 합계 계산
3. 누적 합계를 횟수로 나눠 누적 평균 계산

**핵심 개념**:
- 대수의 법칙: n이 커질수록 표본평균 → 모평균
- 주사위 기댓값 = (1+2+3+4+5+6)/6 = 3.5

**관찰 포인트**:
- 초반(n=10): 평균이 크게 변동
- n=100 이후: 3.5 근처로 안정화
- n=2000: 거의 3.5에 수렴

---

## Q3. 중심극한정리 확인 ⭐⭐

**문제**: 지수분포(평균=5)에서 표본 크기 50으로 500번 추출하여 표본평균의 분포가 정규분포에 가까워지는 것을 확인하세요.

In [ ]:
# Q3 정답
np.random.seed(42)

# 모집단: 지수분포 (평균=5)
population = np.random.exponential(scale=5, size=100000)
sample_size = 50
n_samples = 500

# 모집단 통계
pop_mean = np.mean(population)
pop_std = np.std(population)

print("모집단 정보 (지수분포)")
print("=" * 50)
print(f"모평균: {pop_mean:.4f}")
print(f"모표준편차: {pop_std:.4f}")
print(f"이론적 표준오차: {pop_std/np.sqrt(sample_size):.4f}")

In [ ]:
# 표본평균 500개 생성
sample_means = []
for _ in range(n_samples):
    sample = np.random.choice(population, size=sample_size, replace=True)
    sample_means.append(np.mean(sample))

# 표본평균 분포 통계
print(f"\n표본평균 분포 (n={sample_size}, {n_samples}개 표본)")
print("=" * 50)
print(f"표본평균들의 평균: {np.mean(sample_means):.4f}")
print(f"표본평균들의 표준편차 (SE): {np.std(sample_means):.4f}")
print(f"이론적 SE: {pop_std/np.sqrt(sample_size):.4f}")

In [ ]:
# 시각화: 모집단 vs 표본평균 분포
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=['모집단 (지수분포)', '표본평균 분포 (CLT)'])

# 모집단 분포
fig.add_trace(go.Histogram(x=population[:5000], nbinsx=50, name='모집단',
                           marker_color='blue', opacity=0.7), row=1, col=1)

# 표본평균 분포
fig.add_trace(go.Histogram(x=sample_means, nbinsx=30, name='표본평균',
                           marker_color='red', opacity=0.7), row=1, col=2)

fig.update_layout(height=400, width=900,
                  title_text='중심극한정리: 비정규 모집단에서 정규분포 표본평균')
fig.show()

print("\n왼쪽: 지수분포 (오른쪽 꼬리)")
print("오른쪽: 표본평균 분포 (종 모양 - 정규분포!)")

In [ ]:
# 검증: 정규성 검정
_, p_value = stats.shapiro(sample_means)
print(f"Shapiro-Wilk 정규성 검정 p-value: {p_value:.4f}")
if p_value > 0.05:
    print("검증 통과! 표본평균 분포가 정규분포를 따름 (CLT 확인)")

### 💡 Q3 풀이 설명

**접근 방법**:
1. 지수분포 모집단 생성 (비정규분포)
2. 표본 크기 50으로 500번 추출
3. 각 표본의 평균 계산
4. 표본평균들의 분포 확인

**핵심 개념**:
- 중심극한정리: 모집단 분포와 무관하게 n≥30이면 표본평균은 정규분포
- 지수분포는 오른쪽으로 치우친 비대칭 분포
- 하지만 표본평균의 분포는 종 모양 (정규분포)

**실무 의미**:
- 모집단 분포를 몰라도 표본평균으로 추론 가능
- t-검정, 신뢰구간 등 통계적 방법론의 이론적 근거

---

## Q4. Type I/II 오류 이해 ⭐⭐

**문제**: 다음 시나리오에서 발생한 오류 유형을 판단하세요.

In [ ]:
# Q4 정답

# 오류 유형 분석
scenarios = pd.DataFrame({
    '시나리오': [
        '1. 마케팅 캠페인 (효과 있음 → 없다고 판단)',
        '2. 신약 승인 (효과 없음 → 있다고 판단)',
        '3. 품질검사 (불량품 → 정상 통과)'
    ],
    '실제 상태': ['효과 있음 (H0 거짓)', '효과 없음 (H0 참)', '불량 (H0 거짓)'],
    '검정 결론': ['효과 없음 (H0 채택)', '효과 있음 (H0 기각)', '정상 (H0 채택)'],
    '오류 유형': ['Type II 오류 (β)', 'Type I 오류 (α)', 'Type II 오류 (β)'],
    '결과': [
        '좋은 캠페인을 놓침',
        '효과 없는 약이 시장에 출시',
        '불량품이 고객에게 전달'
    ]
})

print("시나리오별 오류 유형 분석")
print("=" * 80)
print(scenarios.to_string(index=False))

In [ ]:
# 오류 유형 요약 테이블
error_matrix = pd.DataFrame({
    '실제\\판단': ['H0 참 (효과 없음)', 'H0 거짓 (효과 있음)'],
    'H0 기각 (효과 있다)': ['Type I (α) - 가짜 양성', '올바른 결정 (Power)'],
    'H0 채택 (효과 없다)': ['올바른 결정', 'Type II (β) - 가짜 음성']
})

print("\n오류 유형 매트릭스")
print("=" * 60)
print(error_matrix.to_string(index=False))

### 💡 Q4 풀이 설명

**접근 방법**:
1. 각 시나리오의 "실제 상태" 확인
2. "검정 결론"과 비교
3. 오류 유형 판단

**핵심 개념**:
- **Type I 오류 (α)**: H0가 참인데 기각 → "있는데 있다고 (잘못)" → False Positive
- **Type II 오류 (β)**: H0가 거짓인데 채택 → "있는데 없다고 (잘못)" → False Negative

**시나리오별 해설**:
1. 캠페인이 실제로 효과 있음 → 없다고 판단 → Type II (놓침)
2. 약이 실제로 효과 없음 → 있다고 판단 → Type I (거짓 승인)
3. 제품이 실제로 불량 → 정상 통과 → Type II (놓침)

---

## Q5. Cohen's d 계산 ⭐⭐⭐

**문제**: 두 그룹의 데이터로 Cohen's d를 계산하고 효과 크기를 해석하세요.

In [ ]:
# Q5 정답
group_a = [85, 88, 90, 92, 78, 82, 85, 89, 91, 87]
group_b = [95, 92, 98, 88, 96, 94, 90, 97, 93, 91]

# 기술통계
mean_a = np.mean(group_a)
mean_b = np.mean(group_b)
std_a = np.std(group_a, ddof=1)
std_b = np.std(group_b, ddof=1)

print("그룹별 기술통계")
print("=" * 40)
print(f"그룹 A: 평균={mean_a:.2f}, 표준편차={std_a:.2f}")
print(f"그룹 B: 평균={mean_b:.2f}, 표준편차={std_b:.2f}")

In [ ]:
# Cohen's d 계산 함수
def cohens_d(group1, group2):
    """두 그룹 간 Cohen's d 계산"""
    n1, n2 = len(group1), len(group2)
    var1 = np.var(group1, ddof=1)
    var2 = np.var(group2, ddof=1)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    
    return (np.mean(group1) - np.mean(group2)) / pooled_std

# Cohen's d 계산
d = cohens_d(group_b, group_a)  # B - A (B가 더 크므로 양수)

print(f"\nCohen's d = {d:.4f}")

# 효과 크기 해석
if abs(d) < 0.2:
    interpretation = "무시할 수 있는 효과"
elif abs(d) < 0.5:
    interpretation = "작은 효과 (Small)"
elif abs(d) < 0.8:
    interpretation = "중간 효과 (Medium)"
else:
    interpretation = "큰 효과 (Large)"

print(f"효과 크기 해석: {interpretation}")

In [ ]:
# 검증: d가 대략 1.5 근처인지 확인
assert 1.0 < d < 2.0, "Cohen's d 계산 오류"
print(f"검증 통과! d = {d:.4f}는 큰 효과(Large)에 해당")

### 💡 Q5 풀이 설명

**접근 방법**:
1. 각 그룹의 평균, 분산 계산
2. Pooled 표준편차 계산
3. Cohen's d = (M1 - M2) / pooled_std

**핵심 공식**:
```
pooled_std = sqrt(((n1-1)*s1² + (n2-1)*s2²) / (n1+n2-2))
Cohen's d = (M1 - M2) / pooled_std
```

**해석 기준 (Cohen, 1988)**:
- d < 0.2: 무시할 수 있는 효과
- 0.2 ≤ d < 0.5: 작은 효과
- 0.5 ≤ d < 0.8: 중간 효과
- d ≥ 0.8: 큰 효과

**실무 의미**:
- d ≈ 1.5는 매우 큰 효과
- 두 그룹 분포가 거의 겹치지 않음

---

## Q6. 표본 크기 결정 ⭐⭐⭐

**문제**: 중간 효과(d=0.5)를 탐지하기 위해 α=0.05, Power=0.90일 때 필요한 표본 크기를 계산하세요.

In [ ]:
# Q6 정답
from statsmodels.stats.power import TTestIndPower

# Power 분석 객체 생성
power_analysis = TTestIndPower()

# 파라미터 설정
effect_size = 0.5   # 중간 효과
alpha = 0.05        # 유의수준
power = 0.90        # 목표 검정력 (90%)

# 표본 크기 계산
sample_size = power_analysis.solve_power(
    effect_size=effect_size,
    alpha=alpha,
    power=power,
    alternative='two-sided'
)

print("표본 크기 계산 결과")
print("=" * 50)
print(f"효과 크기 (Cohen's d): {effect_size}")
print(f"유의수준 (α): {alpha}")
print(f"목표 검정력: {power*100:.0f}%")
print(f"\n필요 표본 크기: 각 그룹 {int(np.ceil(sample_size))}명")
print(f"총 필요 인원: {int(np.ceil(sample_size)) * 2}명")

In [ ]:
# 80% vs 90% 검정력 비교
sample_80 = power_analysis.solve_power(effect_size=0.5, alpha=0.05, power=0.80)
sample_90 = power_analysis.solve_power(effect_size=0.5, alpha=0.05, power=0.90)

print("\n검정력별 필요 표본 크기 비교")
print("=" * 50)
print(f"80% 검정력: 각 그룹 {int(np.ceil(sample_80))}명")
print(f"90% 검정력: 각 그룹 {int(np.ceil(sample_90))}명")
print(f"\n10%p 검정력 향상에 {int(np.ceil(sample_90) - np.ceil(sample_80))}명 추가 필요")

In [ ]:
# 검증
assert 80 < sample_size < 90, "표본 크기 계산 오류"
print(f"검증 통과! 90% 검정력을 위해 각 그룹 약 86명 필요")

### 💡 Q6 풀이 설명

**접근 방법**:
1. `TTestIndPower` 객체 생성
2. `solve_power()` 메서드로 필요 표본 크기 계산

**핵심 개념**:
- Power 분석: 효과를 탐지할 확률 계산
- 90% 검정력 > 80% 검정력 → 더 많은 표본 필요

**실무 팁**:
- 80%: 최소 권장 검정력
- 90%: 좀 더 확실한 결과를 원할 때
- 비용/시간 vs 검정력 트레이드오프 고려

---

## Q7. 검정력 분석 ⭐⭐⭐

**문제**: 효과 크기가 0.3, 0.5, 0.8일 때 표본 크기에 따른 검정력 변화를 비교하는 그래프를 그리세요.

In [ ]:
# Q7 정답
from statsmodels.stats.power import TTestIndPower

power_analysis = TTestIndPower()

# 설정
effect_sizes = [0.3, 0.5, 0.8]
n_range = np.arange(10, 201, 5)
alpha = 0.05

# 각 효과 크기별 검정력 계산
results = {}
for d in effect_sizes:
    powers = []
    for n in n_range:
        p = power_analysis.solve_power(effect_size=d, alpha=alpha, nobs1=n)
        powers.append(p)
    results[f'd={d}'] = powers

# 데이터프레임 생성
df_power = pd.DataFrame(results, index=n_range)
df_power.index.name = '표본 크기'

print("표본 크기별 검정력 (일부)")
print(df_power.loc[[20, 50, 100, 200]].round(3))

In [ ]:
# 시각화
fig = go.Figure()

colors = ['blue', 'green', 'red']
for i, d in enumerate(effect_sizes):
    fig.add_trace(go.Scatter(
        x=n_range,
        y=results[f'd={d}'],
        mode='lines',
        name=f'효과크기 d={d}',
        line=dict(color=colors[i], width=2)
    ))

# 80% 검정력 기준선
fig.add_hline(y=0.80, line_dash="dash", line_color="black",
              annotation_text="목표 검정력 80%")

fig.update_layout(
    title='효과 크기별 표본 크기-검정력 관계 (α=0.05)',
    xaxis_title='각 그룹 표본 크기',
    yaxis_title='검정력 (Power)',
    height=500, width=800,
    legend=dict(x=0.7, y=0.3)
)

fig.show()

In [ ]:
# 각 효과 크기별 80% 검정력 달성 표본 크기
print("\n80% 검정력 달성에 필요한 표본 크기")
print("=" * 50)
for d in effect_sizes:
    n = power_analysis.solve_power(effect_size=d, alpha=0.05, power=0.80)
    print(f"효과크기 d={d}: 각 그룹 {int(np.ceil(n))}명")

### 💡 Q7 풀이 설명

**접근 방법**:
1. 효과 크기별 반복문
2. 표본 크기 범위에서 검정력 계산
3. Plotly로 3개 선 그래프 생성

**관찰 포인트**:
- 효과 크기가 클수록 적은 표본으로 80% 검정력 달성
- d=0.8: 약 26명/그룹
- d=0.5: 약 64명/그룹
- d=0.3: 약 176명/그룹

**실무 의미**:
- 작은 효과를 탐지하려면 큰 표본 필요
- 예산/시간 제약 시 탐지 가능한 최소 효과 크기 재검토

---

## Q8. 중심극한정리 심화 ⭐⭐⭐⭐

**문제**: 베르누이 분포(성공 확률 p=0.3)에서 표본 크기별 표본평균 분포를 비교하세요.

In [ ]:
# Q8 정답
np.random.seed(42)

p = 0.3  # 성공 확률
sample_sizes = [5, 15, 30, 100]
n_samples = 1000

# 이론값
theoretical_mean = p
theoretical_var = p * (1 - p)

print("베르누이 분포 이론값")
print("=" * 50)
print(f"모평균: {theoretical_mean}")
print(f"모분산: {theoretical_var}")

In [ ]:
# 각 표본 크기별 표본평균 생성
results = {}

print("\n표본 크기별 표본평균 분포 통계")
print("=" * 70)
print(f"{'n':<6} {'실제 평균':>10} {'이론 평균':>10} {'실제 SE':>10} {'이론 SE':>10}")
print("-" * 70)

for n in sample_sizes:
    sample_means = []
    for _ in range(n_samples):
        # 베르누이 분포: 0 또는 1
        sample = np.random.binomial(1, p, size=n)
        sample_means.append(np.mean(sample))
    
    results[n] = sample_means
    
    # 통계 계산
    actual_mean = np.mean(sample_means)
    actual_se = np.std(sample_means)
    theoretical_se = np.sqrt(theoretical_var / n)
    
    print(f"{n:<6} {actual_mean:>10.4f} {theoretical_mean:>10.4f} {actual_se:>10.4f} {theoretical_se:>10.4f}")

In [ ]:
# 시각화: 4개 서브플롯
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=[f'n = {n}' for n in sample_sizes])

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for n, pos in zip(sample_sizes, positions):
    fig.add_trace(
        go.Histogram(x=results[n], nbinsx=25, name=f'n={n}',
                     marker_color='steelblue', opacity=0.7),
        row=pos[0], col=pos[1]
    )
    
    # 평균선 추가
    fig.add_vline(x=p, line_dash="dash", line_color="red",
                  row=pos[0], col=pos[1])

fig.update_layout(height=600, width=900,
                  title_text='베르누이 분포(p=0.3)의 표본평균 분포 (CLT)',
                  showlegend=False)
fig.show()

print("\n관찰 포인트:")
print("- n=5: 이산적, 정규분포 아님")
print("- n=15: 종 모양에 가까워짐")
print("- n=30: 거의 정규분포")
print("- n=100: 완전한 정규분포, SE 매우 작음")

### 💡 Q8 풀이 설명

**접근 방법**:
1. 베르누이 분포 (0 또는 1)에서 표본 추출
2. 각 표본 크기별로 1000개의 표본평균 생성
3. 이론값과 실제값 비교

**핵심 개념**:
- 베르누이: 가장 단순한 이산분포 (0/1)
- 평균 = p = 0.3
- 분산 = p(1-p) = 0.21
- SE = sqrt(p(1-p)/n)

**CLT 관찰**:
- n=5: 표본평균이 0, 0.2, 0.4, 0.6, 0.8, 1.0 중 하나 (이산적)
- n=30: 중심극한정리 적용, 연속적 정규분포
- n=100: 표준오차가 매우 작아져 좁은 분포

---

## Q9. A/B 테스트 설계 ⭐⭐⭐⭐

**문제**: 전환율 3% → 3.5% 개선을 위한 A/B 테스트를 설계하세요.

In [ ]:
# Q9 정답
from statsmodels.stats.power import NormalIndPower

# 조건 설정
baseline_rate = 0.03    # 현재 전환율 3%
target_rate = 0.035     # 목표 전환율 3.5%
daily_traffic = 10000   # 일일 트래픽

# 상대적 개선율
relative_lift = (target_rate - baseline_rate) / baseline_rate

print("A/B 테스트 설계")
print("=" * 50)
print(f"현재 전환율: {baseline_rate:.1%}")
print(f"목표 전환율: {target_rate:.1%}")
print(f"상대적 개선율: {relative_lift:.1%}")
print(f"일일 트래픽: {daily_traffic:,}명")

In [ ]:
# 1. 효과 크기 계산 (Cohen's h - 비율 비교용)
effect_size_h = 2 * (np.arcsin(np.sqrt(target_rate)) - np.arcsin(np.sqrt(baseline_rate)))

print(f"\n1. 효과 크기 (Cohen's h): {effect_size_h:.4f}")

# 2. 80% 검정력 표본 크기
power_analysis = NormalIndPower()

sample_80 = power_analysis.solve_power(
    effect_size=effect_size_h,
    alpha=0.05,
    power=0.80,
    alternative='two-sided'
)

total_sample_80 = int(np.ceil(sample_80)) * 2
days_80 = int(np.ceil(total_sample_80 / daily_traffic))

print(f"\n2. 80% 검정력 기준")
print(f"   필요 표본 크기: 각 그룹 {int(np.ceil(sample_80)):,}명")
print(f"   총 필요 인원: {total_sample_80:,}명")
print(f"   예상 테스트 기간: {days_80}일")

In [ ]:
# 3. 90% 검정력 표본 크기
sample_90 = power_analysis.solve_power(
    effect_size=effect_size_h,
    alpha=0.05,
    power=0.90,
    alternative='two-sided'
)

total_sample_90 = int(np.ceil(sample_90)) * 2
days_90 = int(np.ceil(total_sample_90 / daily_traffic))

print(f"\n3. 90% 검정력 기준")
print(f"   필요 표본 크기: 각 그룹 {int(np.ceil(sample_90)):,}명")
print(f"   총 필요 인원: {total_sample_90:,}명")
print(f"   예상 테스트 기간: {days_90}일")

# 추가 필요 일수
additional_days = days_90 - days_80
print(f"\n90% 검정력을 위해 {additional_days}일 추가 필요")

### 💡 Q9 풀이 설명

**접근 방법**:
1. 비율 비교용 효과 크기(Cohen's h) 계산
2. NormalIndPower로 표본 크기 계산
3. 일일 트래픽으로 테스트 기간 산출

**핵심 공식**:
```
Cohen's h = 2 * (arcsin(sqrt(p2)) - arcsin(sqrt(p1)))
```

**결과 해석**:
- 3% → 3.5%는 작은 효과 (h ≈ 0.03)
- 작은 효과를 탐지하려면 큰 표본 필요
- 80% 검정력: 약 2-3주
- 90% 검정력: 약 3-4주

---

## Q10. 종합 분석 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 신규 앱 기능의 사용자 체류 시간 증가 효과를 검증하세요.

In [ ]:
# Q10 정답
np.random.seed(2025)

# 조건 설정
baseline_mean = 5.0  # 기존 평균 체류 시간 (분)
expected_mean = 5.5  # 목표 평균 (10% 증가)
std_dev = 2.0        # 표준편차

print("신규 앱 기능 효과 검증 실험")
print("=" * 60)
print(f"기존 평균 체류 시간: {baseline_mean}분")
print(f"목표 평균 체류 시간: {expected_mean}분 (+{(expected_mean-baseline_mean)/baseline_mean:.0%})")
print(f"표준편차: {std_dev}분")

In [ ]:
# 1. 효과 크기 계산
effect_size = (expected_mean - baseline_mean) / std_dev

print(f"\n1. 효과 크기 분석")
print("-" * 40)
print(f"Cohen's d = ({expected_mean} - {baseline_mean}) / {std_dev} = {effect_size:.2f}")

if effect_size < 0.2:
    size_label = "매우 작음"
elif effect_size < 0.5:
    size_label = "작음 (Small)"
elif effect_size < 0.8:
    size_label = "중간 (Medium)"
else:
    size_label = "큼 (Large)"
print(f"효과 크기 해석: {size_label}")

In [ ]:
# 2. 표본 크기 계산
from statsmodels.stats.power import TTestIndPower

power_analysis = TTestIndPower()
required_n = power_analysis.solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.80,
    alternative='two-sided'
)

n_per_group = int(np.ceil(required_n))

print(f"\n2. 표본 크기 계산 (80% 검정력)")
print("-" * 40)
print(f"필요 표본 크기: 각 그룹 {n_per_group}명")
print(f"총 필요 인원: {n_per_group * 2}명")

In [ ]:
# 3. 가상 데이터 생성 및 t-검정
np.random.seed(2025)

# 대조군 (기존 기능)
control = np.random.normal(baseline_mean, std_dev, n_per_group)

# 실험군 (신규 기능)
treatment = np.random.normal(expected_mean, std_dev, n_per_group)

# t-검정 수행
t_stat, p_value = stats.ttest_ind(treatment, control)

print(f"\n3. t-검정 결과")
print("-" * 40)
print(f"대조군 평균: {np.mean(control):.2f}분")
print(f"실험군 평균: {np.mean(treatment):.2f}분")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 결론
alpha = 0.05
if p_value < alpha:
    print(f"\n결론: p-value ({p_value:.4f}) < α ({alpha})")
    print("       → 귀무가설 기각. 신규 기능이 체류 시간 증가에 유의한 효과!")
else:
    print(f"\n결론: p-value ({p_value:.4f}) >= α ({alpha})")
    print("       → 귀무가설 채택. 통계적으로 유의한 차이 없음.")

In [ ]:
# 4. 시각화
fig = go.Figure()

# 대조군 히스토그램
fig.add_trace(go.Histogram(
    x=control,
    name='대조군 (기존)',
    opacity=0.7,
    marker_color='blue'
))

# 실험군 히스토그램
fig.add_trace(go.Histogram(
    x=treatment,
    name='실험군 (신규)',
    opacity=0.7,
    marker_color='red'
))

# 평균선 추가
fig.add_vline(x=np.mean(control), line_dash="dash", line_color="blue",
              annotation_text=f"대조군 평균: {np.mean(control):.2f}")
fig.add_vline(x=np.mean(treatment), line_dash="dash", line_color="red",
              annotation_text=f"실험군 평균: {np.mean(treatment):.2f}")

fig.update_layout(
    title='신규 앱 기능 A/B 테스트: 체류 시간 분포 비교',
    xaxis_title='체류 시간 (분)',
    yaxis_title='빈도',
    barmode='overlay',
    height=450, width=800
)

fig.show()

In [ ]:
# 5. 종합 분석 보고서
# 실제 효과 크기 계산
actual_d = (np.mean(treatment) - np.mean(control)) / np.sqrt(
    ((len(control)-1)*np.var(control, ddof=1) + (len(treatment)-1)*np.var(treatment, ddof=1)) / 
    (len(control)+len(treatment)-2)
)

# 실제 검정력 계산
actual_power = power_analysis.solve_power(
    effect_size=actual_d,
    alpha=0.05,
    nobs1=n_per_group,
    alternative='two-sided'
)

print("\n" + "=" * 60)
print("5. 종합 분석 보고서")
print("=" * 60)

report = pd.DataFrame({
    '지표': ['표본 크기 (각 그룹)', 'p-value', '효과 크기 (Cohen\'s d)', '검정력', '결론'],
    '계획': [f'{n_per_group}명', f'< 0.05', f'{effect_size:.2f}', '80%', '-'],
    '실제': [f'{n_per_group}명', f'{p_value:.4f}', f'{actual_d:.2f}', f'{actual_power:.1%}', 
            '유의함' if p_value < 0.05 else '유의하지 않음']
})

print(report.to_string(index=False))

print("\n" + "=" * 60)
print("최종 결론")
print("=" * 60)
if p_value < 0.05:
    improvement = (np.mean(treatment) - np.mean(control)) / np.mean(control) * 100
    print(f"신규 앱 기능은 사용자 체류 시간을 약 {improvement:.1f}% 증가시킵니다.")
    print(f"이는 통계적으로 유의한 결과이며 (p={p_value:.4f}),")
    print(f"효과 크기는 {size_label}입니다 (d={actual_d:.2f}).")
    print("\n권장: 신규 기능 전체 출시 진행")
else:
    print("신규 앱 기능의 효과가 통계적으로 유의하지 않습니다.")
    print("\n권장: 기능 개선 또는 추가 테스트 필요")

### 💡 Q10 풀이 설명

**접근 방법**:
1. 효과 크기 계산 (d = 차이/표준편차)
2. Power 분석으로 표본 크기 결정
3. 가상 데이터 생성 (시뮬레이션)
4. t-검정 수행
5. 결과 시각화 및 해석

**핵심 개념 통합**:
- Cohen's d로 효과 크기 정량화
- Power 분석으로 적절한 표본 크기 결정
- t-검정으로 통계적 유의성 확인
- p-value와 효과 크기 함께 보고

**실무 체크리스트**:
- [ ] 탐지할 최소 효과 크기 정의
- [ ] 필요 표본 크기 계산 (Power 분석)
- [ ] 데이터 수집 및 무작위 배정
- [ ] 적절한 검정 수행 (t-test, chi-square 등)
- [ ] p-value + 효과 크기 + 신뢰구간 함께 보고
- [ ] 비즈니스 관점 해석 및 의사결정

---

## 📊 최종 학습 정리

### Part 1: 기초 개념

| 개념 | 핵심 포인트 | 공식/조건 |
|------|------------|----------|
| 대수의 법칙 | n↑ → 표본평균 → 모평균 | 많이 관찰하면 수렴 |
| 중심극한정리 | 표본평균은 정규분포 | n ≥ 30 |
| 표준오차 | 표본평균의 불확실성 | SE = σ/√n |

### Part 2: 심화 개념

| 개념 | 정의 | 실무 활용 |
|------|------|----------|
| Type I 오류 | 없는데 있다고 | α로 통제 (0.05) |
| Type II 오류 | 있는데 없다고 | 검정력으로 관리 |
| 검정력 | 효과 발견 확률 | 80% 이상 권장 |
| Cohen's d | 효과 크기 | 0.2/0.5/0.8 기준 |

### 실무 적용 순서

```
1. 효과 크기 정의 (비즈니스 관점)
   ↓
2. Power 분석 → 표본 크기 결정
   ↓
3. 실험 수행 및 데이터 수집
   ↓
4. 통계 검정 (t-test 등)
   ↓
5. 결과 해석 (p-value + 효과 크기)
```